# How to work with maps with follium

## Locate the computational domain on a map

In [1]:
from os import environ, path
import sys
import numpy as np
try:
    import pyproj
    import folium
except ImportError:
    raise Exception("Warning: pyproj and folium are mandatory for that notebook")
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
from data_manip.extraction.telemac_file import TelemacFile
from postel.plot2d import *

# File we are going to use
file_name = path.join(environ['HOMETEL'], 
                      'examples', 
                      'telemac2d', 
                      'tide',
                      'f2d_tide-jmj_type.slf')

# Initalisaing Telemac file reader
res = TelemacFile(file_name)

# Coordoate projections
wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
lamb1=pyproj.Proj("+init=EPSG:27561")

# Conversion from Lambert Nord to WGS84
res.tri.x, res.tri.y = pyproj.transform(lamb1, wgs84, res.tri.x, res.tri.y)

# Define location of mesh
xlim = [np.min(res.tri.x), np.max(res.tri.x)]
ylim = [np.min(res.tri.y), np.max(res.tri.y)]
x_center = (xlim[0] + xlim[1])/2.
y_center = (ylim[0] + ylim[1])/2.

top_left_point = [ylim[1], xlim[0]]
bottom_left_point = [ylim[0], xlim[0]]
top_right_point = [ylim[1], xlim[1]]
bottom_right_point = [ylim[0], xlim[1]]

height = ylim[1] - ylim[0]
width = xlim[1] - xlim[0]

# Creation of raster layer for map overlay
carte_wms = folium.Map(
    location = [y_center, x_center], # [latittude, longitude]  
    width = '100%',
    height= '100%',
    zoom_start = 10,
    max_native_zoom = None,
    control_scale = True,
    disable_3d = True,
    zoom_control = True, 
    )

folium.Marker(top_left_point).add_to(carte_wms)
folium.Marker(bottom_right_point).add_to(carte_wms)
folium.Marker(bottom_left_point).add_to(carte_wms)
folium.Marker(top_right_point).add_to(carte_wms)

carte_wms

In [2]:
# Ajout wms raster layer
rasterlayer = folium.raster_layers.WmsTileLayer(
    url = 'http://noeyyrh6.hadam.hadroot.edf.fr:6080/arcgis/services/EXT/BDORTHO_France/MapServer/WMSServer',
    name = 'test',
    fmt = 'image/png',
    version = "1.3.0",
    layers = '0',
    attr = u'EDF',
    transparent = False,
    overlay = True,
    control = True)

rasterlayer.add_to(carte_wms)

carte_wms.fit_bounds((bottom_left_point, top_right_point), padding_top_left=top_left_point, 
                     padding_bottom_right=bottom_right_point, max_zoom=100)
carte_wms

In [3]:
del res